# Grids and Matrices

## Moving in a grid
Key approaches are: 
- factor out validation as a helper function, to avoid cluttering the main algorithm with checks.
- create a directions array that includes all possible changes to the current position to reach neighbouring cells.

#### Chess Moves
Create a function that returns possible moves for a king, queen or knight chess piece, given a board of size n x n, where 0 denotes an empty cell and 1 denotes an occupied one, and given starting coordinates.

In [77]:
# I factor out a validation factor, which checks if a given move would be within the confines of the board and to an unoccupied cell.
# This means I don't have to keep checking within the main algorithm.
def is_valid(board, r, c):
    n = len(board)
    if 0 <= r < n and 0 <= c < n:
        if board[r][c] == 0:
            return True
    return False


def chess_moves(board, piece, r, c):
    moves = []
    potential_moves = []

    if piece.lower() == "knight":
        # list comprehensions used to create directions arrays
        directions = [(rd, cd) for rd in [-2, 2] for cd in [-1, 1]] + [(rd, cd) for rd in [-1, 1] for cd in [-2, 2]]
    else: # king and queen both move in the same directions
        directions = [(rd, cd) for rd in [-1, 0, 1] for cd in [-1, 0, 1] if (rd != 0 or cd != 0)]  

    for rd, cd in directions:
        if piece.lower() == "queen": # As queen can travel across multiple spaces we need a loop to take her as far as 
            rm, cm = r+rd, c+cd      # she can go in each direction before hitting another piece or the end of the board.
            while is_valid(board, rm, cm):
                moves.append([rm, cm])
                rm+=rd
                cm+=cd
        else:
            if is_valid(board, r+rd, c+cd):
                moves.append([r+rd, c+cd])

    return moves

In [79]:
board = [[0, 0, 0, 1, 0, 0],
         [0, 1, 1, 1, 0, 0],
         [0, 1, 0, 1, 1, 0],
         [1, 1, 1, 1, 0, 0],
         [0, 0, 0, 0, 0, 0],
         [0, 1, 0, 0, 0, 0]]

chess_moves(board, "king", 3, 5)

[[2, 5], [3, 4], [4, 4], [4, 5]]

#### Queen's reach
A board has a number of queens on it, indicated by ones. Return a board that uses zeros to denote spaces that are safe from the queens.

In [162]:
# One approach is to use the chess_moves algorithm to determine all of the potential moves for each queen. 
# This may be inefficient if there's a high concentration of queens. It may be better to go through each cell and check for threats.

def queens_reach(board):
    n = len(board)
    updated_board = [row.copy for row in board] # NB if you copy the grid, updating one will update the other. To create a new grid: [[0]*n for _ in range(n)]
    for r in range(n):
        for c in range(n):
            if board[r][c] == 1:
                updated_board[r][c] = 1
                moves = chess_moves(board, "queen", r, c)
                for rm, cm in moves:
                    updated_board[rm][cm] = 1
    return updated_board

In [164]:
board = [[0]*6 for _ in range(6)]
board[1][1] = 1
board[3][4] = 1
board[4][4] = 1
board

[[0, 0, 0, 0, 0, 0],
 [0, 1, 0, 0, 0, 0],
 [0, 0, 0, 0, 0, 0],
 [0, 0, 0, 0, 1, 0],
 [0, 0, 0, 0, 1, 0],
 [0, 0, 0, 0, 0, 0]]

In [166]:
queens_reach(board)

[[1, 1, 1, 0, 1, 0],
 [1, 1, 1, 1, 1, 1],
 [1, 1, 1, 1, 1, 1],
 [1, 1, 1, 1, 1, 1],
 [1, 1, 1, 1, 1, 1],
 [0, 1, 1, 1, 1, 1]]

##### Considerations
Each queen takes O(4n) so we get 4n x num_queens. The number of queens ranges from 0 to nxn; however, in the worst case, the queen will meet obstacles (other queens) on every side, so each queen would check only 8 cells. That makes the worst case O(8n^2) or O(n^2)

If we go cell by cell, each cell requires 4n checks, so the total is 4n x nxn = 4n^3. This approach would certainly be worse 
unless there are more queens than empty cells.

There may be another approach that would check for queens row by row, then column by column, then along the diagonals. Each dimension would be nxn so the total would be 4nxn. This would take half the time complexity of the worst case of the option coded, but more than the best case.

#### Spiral order
Given a +ve, odd integer, n, return an nxn grid, filled with every number from $0$ to $n^{2}-1$, spiralling clockwise down from the center.

In [284]:
# Validation function to check if coordinates are empty, should a turn be made
def turn(grid, r, c):
    return grid[r][c] == None

def spiral_order(n):
    grid = [[None]*n for _ in range(n)]
    dir = [[1,0], [0,-1], [-1,0], [0,1]] # Directions array. The function will need to cycle through these in order.
    d = -1 # d is a counter that determines which direction will be used.
    r, c = n//2, n//2
    for i in range(n**2):       
        grid[r][c] = i
        if turn(grid, r+dir[(d+1)%4][0], c+dir[(d+1)%4][1]): # Checks if making a turn is a valid move
            d = (d+1)%4 # Changes direction
        r += dir[d][0]
        c += dir[d][1]                   
    return grid    


In [286]:
spiral_order(5)

[[16, 17, 18, 19, 20],
 [15, 4, 5, 6, 21],
 [14, 3, 0, 7, 22],
 [13, 2, 1, 8, 23],
 [12, 11, 10, 9, 24]]

In [288]:
# The spiral always ends in the bottom right corner `grid[n][n]`.
# A conceptually easier approach is to start there and work backwards, using the edges of the grid to guide the movement.
def valid_move(grid, r, c):
    n = len(grid)
    if 0 <= r < n and 0 <= c < n:
        if grid[r][c] == 0:
            return True
    return False

def spiral_order_reverse(n):
    grid = [[0]*n for _ in range(n)]
    directions = [[-1,0],[0,-1],[1,0],[0,1]]
    d = 0
    r, c = n-1, n-1
    for i in range(n**2-1, 0, -1):
        grid[r][c] = i
        if not valid_move(grid, r+directions[d][0], c+directions[d][1]):
            d = (d + 1) % 4 # Changes direction
        r += directions[d][0]
        c += directions[d][1]
    return grid

In [290]:
spiral_order_reverse(5)

[[16, 17, 18, 19, 20],
 [15, 4, 5, 6, 21],
 [14, 3, 0, 7, 22],
 [13, 2, 1, 8, 23],
 [12, 11, 10, 9, 24]]

##### Analysis
Time complexity is $O(n^{2})$ for both models: the validation function takes constant time and is applied to each cell in the grid once. 
Spatial complexity is also $O(n^{2})$.